In [51]:
## Imports ##
from math import factorial
import numpy as np
import itertools

In [33]:
## Funciones auxiliares##
## Función s_b(perm)##
def extension_derecha(perm, b):
    for i in range(len(perm)):
        if perm[i] >= b:
            perm[i] = perm[i] + 1
    perm.append(b)
    return perm

## Función comprobatoria de la relación de orden##

def relacion_equivalencia_orden(perm1, perm2):
    m = len(perm1)
    n = len(perm2)
    if m != n:
        return False
    for i in range(m-1):
        for j in range(i+1,m):
            if perm1[i] < perm1[j] and perm2[i] >= perm2[j]:
                return False
            if perm1[i] > perm1[j] and perm2[i] <= perm2[j]:
                return False
            if perm1[i] == perm1[j] and perm2[i] != perm2[j]:
                return False
    return True

## Relacion para permutaciones abreviadas ##
## Nota: Esta función asume que el rango de valores de los inputs es [n] ##
def relacion_equivalencia_shorthand(abreviada, original):
    m = len(abreviada)
    n = len(original)
    if m != n -1 or abreviada != original[0:m]:
        return False
    if original[m] in abreviada:
        return False
    return True

## Extracción de todos los n factores de una permutación dada ##
def extraer_n_factores(perm, n):
    factores = set()
    for i in range(len(perm)-n+1):
        factores.add(tuple(perm[i:i+n]))
    return factores


## Extracción de todos los n factores de una permutación cíclica dada ##
def extraer_factores_ciclicos(perm, n):
    factores = set()
    l = len(perm)
    for i in range(l):
        if i+n > l:
            r = i+n-l
            factores.add(tuple(perm[i:] + perm[0:r]))
        else:
            factores.add(tuple(perm[i:i+n])) 
    return factores

## Convertir u-palabra en un u-ciclo ##
def u_palabra_to_u_ciclo(word, n):
    word = word[0:len(word)-n+1]
    result=[]
    m = len(word)
    for i in range(m):
        count = 0
        for j in range(m):
            if word[i] >= word[j]:
                count += 1
        result.append(count)
    return result

def aux_equivalence(a, b):
    if a <= b: return 0
    else: return 1
    
## Permutacion expresada como matriz triangular inferior ##
def permutacion_to_matriz(perm):
    
    rango = len(perm)-1
    matriz = np.zeros((rango, rango), int)
    
    for i in range(rango):
        #print(i)
        for j in range(i+1):
            matriz[i][j] = aux_equivalence(perm[j],perm[i+1])
    return matriz

def extension_matrix(matriz):
    rango = len(matriz)
    for i in range(rango-1):
        #print(i)
        for j in range(i+1):
            #print(j)
            matriz[i][j] = matriz[i+1][j+1]
    for i in range(len(matriz)):
        matriz[len(matriz)-1][i] = -1
    return matriz
    

In [34]:
## Celda de testing

permutacion_to_matriz([2,3,5,4,1])

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 1, 1, 1]])

In [35]:
extension_matrix(permutacion_to_matriz([2,3,5,4,1]))

array([[ 0,  0,  0,  0],
       [ 0,  1,  0,  0],
       [ 1,  1,  1,  0],
       [-1, -1, -1, -1]])

In [36]:
# Algoritmo comprobatorio de u-ciclos

def comprobar_u_ciclo(ciclo, n, window_length, relacion):
    permutations = set(list(itertools.permutations([i for i in range(1,n+1)])))
    ventanas = extraer_factores_ciclicos(ciclo, window_length)
    for ventana in ventanas:
        flag = False
        for perm in permutations:
            if relacion(ventana, perm):
                permutations.remove(perm)
                flag = True
                break
        if not flag:
            return False
    return True


In [37]:

## Algoritmo greedy del capítulo 2 ##
def greedy_algoritmo(n):
    perm = []
    k = 0
    ord_isometrico = False
    extended = True
    # Generación de la permutación inicial
    for  i in range(n-1):
        perm.append(i+1)
        
    #Comenzamos a buscar extensiones
    while(1):
        #Check: Si no pudimos extender nos salimos
        if extended is False:
            break
        extended = False
        # Cogemos los símbolos de la permutación en orden
        extensionSet = set(perm)
        factorSet = extraer_n_factores(perm,n)
        # Ejecución default, en la primera iteración siempre falta un factor para llegar a longitud n
        # Por defecto añadimos n a la primera permutación
        if len(perm) < n:
            temp = []
            for i in range(n):
                temp.append(i+1)
            factorSet.add(tuple(temp))
            
        # Vemos si algún factor es orden isométrico
        for element in extensionSet:
            ord_isometrico = False
            for tupla in factorSet:
                # Si es orden iso marcamos True
                if relacion_equivalencia_orden(extension_derecha(perm[k:n+k-1], element),tupla) is True:
                    ord_isometrico = True
                    break
                # Si no es orden iso entonces extender la permutación
            if ord_isometrico is False:
                perm = extension_derecha(perm, element)
                extended = True
                k+=1
                break
    return u_palabra_to_u_ciclo(perm, n)

In [38]:
## Test del algoritmo greedy para n=6 ##

print(greedy_algoritmo(5))
comprobar_u_ciclo(greedy_algoritmo(5), 5, 5, relacion_equivalencia_orden)

[117, 118, 119, 120, 116, 115, 114, 112, 113, 111, 108, 110, 109, 106, 105, 107, 103, 101, 104, 102, 98, 99, 100, 96, 97, 91, 95, 93, 92, 90, 94, 83, 87, 89, 84, 80, 88, 85, 74, 81, 86, 73, 75, 82, 76, 72, 77, 79, 70, 78, 66, 71, 67, 68, 25, 69, 24, 27, 26, 65, 1, 28, 29, 30, 2, 31, 23, 22, 3, 32, 5, 20, 4, 33, 6, 21, 19, 34, 7, 36, 35, 8, 18, 37, 10, 9, 38, 16, 11, 17, 39, 12, 14, 40, 15, 13, 47, 41, 42, 44, 48, 43, 45, 51, 49, 46, 52, 53, 50, 54, 57, 55, 56, 58, 61, 59, 62, 60, 64, 63]


True

In [45]:
## Algoritmo coolex del capítulo 3 ##

def prefijo_no_incremental_longitud(s):
    for i in range(len(s)-1):
        if s[i] < s[i+1]:
                return i+1
    return len(s)

def prefijo_derecha(s, j):
    return s[1:j+1] + [s[0]] + s[j+1:]

def cool_lex_algoritmo(n):
    coolex = [] 
    perm_inicial = [i for i in range(1,n+1)]
    coolex.append(perm_inicial)

    for i in range(np.math.factorial(n)-1):

        s = coolex[-1]
        # Buscamos cardinal prefijo no incremental
        k = prefijo_no_incremental_longitud(s[1:])
        if k <= n and s[0] > s[k]:
            coolex.append(prefijo_derecha(s, k))
        elif k <= n and s[0] < s[k]:
            coolex.append(prefijo_derecha(s, k+1))
        else:
            coolex.append(prefijo_derecha(s,n-1))

    return coolex

In [46]:
cool_lex_algoritmo(3)

[[1, 2, 3], [2, 3, 1], [3, 1, 2], [1, 3, 2], [3, 2, 1], [2, 1, 3]]

In [122]:
# Algoritmo bell-ringer

def circular_substring_bellringer(s, n):
    start = s.index(n)
    for i in range (len(s)):
        if s[(start+1+i)%len(s)] != s[(start+i)%len(s)]-1:
            break
    return s[(start+i)%len(s)]

def bell_ringer_algoritmo(n):
    bell_ringer = [] 
    perm_inicial = [n-i for i in range(0,n)]
    n_position = n-1
    # Añadimos las dos primeras permutaciones na y an donde a = an-1...a1
    bell_ringer.append(perm_inicial)
    for i in range(np.math.factorial(n)-1):
        s = bell_ringer[-1]
        m = np.max([s[0],s[-1]])
        k = circular_substring_bellringer(s, n)
        if k-1 <= m <= n-1:
            bell_ringer.append(s[1:n-1] + [s[0]] + [s[n-1]])
        else:
            bell_ringer.append(s[1:n] + [s[0]])
    return bell_ringer

In [130]:
# Generador de u-ciclos abreviados

def generador_u_ciclo(n, algoritmo):
    lista = algoritmo(n-1)
    print(lista)
    u_ciclo = []
    for perm in lista:
        u_ciclo = u_ciclo + [n] + perm
    print(u_ciclo)
    return u_ciclo


In [123]:
print(len(generador_u_ciclo(4, cool_lex_algoritmo)))

[[1, 2, 3], [2, 3, 1], [3, 1, 2], [1, 3, 2], [3, 2, 1], [2, 1, 3]]
24


In [67]:
# Vemos si estamos generando u-ciclos de verdad o no 

comprobar_u_ciclo(generador_u_ciclo(7, cool_lex_algoritmo), 7, 6, relacion_equivalencia_shorthand)

[[1, 2, 3, 4, 5, 6], [2, 3, 1, 4, 5, 6], [3, 1, 2, 4, 5, 6], [1, 3, 2, 4, 5, 6], [3, 2, 4, 1, 5, 6], [2, 3, 4, 1, 5, 6], [3, 4, 2, 1, 5, 6], [4, 2, 1, 3, 5, 6], [2, 1, 4, 3, 5, 6], [1, 2, 4, 3, 5, 6], [2, 4, 1, 3, 5, 6], [4, 1, 2, 3, 5, 6], [1, 4, 2, 3, 5, 6], [4, 2, 3, 1, 5, 6], [2, 4, 3, 1, 5, 6], [4, 3, 1, 2, 5, 6], [3, 1, 4, 2, 5, 6], [1, 3, 4, 2, 5, 6], [3, 4, 1, 2, 5, 6], [4, 1, 3, 2, 5, 6], [1, 4, 3, 2, 5, 6], [4, 3, 2, 5, 1, 6], [3, 2, 4, 5, 1, 6], [2, 3, 4, 5, 1, 6], [3, 4, 2, 5, 1, 6], [4, 2, 3, 5, 1, 6], [2, 4, 3, 5, 1, 6], [4, 3, 5, 2, 1, 6], [3, 4, 5, 2, 1, 6], [4, 5, 3, 2, 1, 6], [5, 3, 2, 1, 4, 6], [3, 2, 1, 5, 4, 6], [2, 1, 3, 5, 4, 6], [1, 2, 3, 5, 4, 6], [2, 3, 1, 5, 4, 6], [3, 1, 2, 5, 4, 6], [1, 3, 2, 5, 4, 6], [3, 2, 5, 1, 4, 6], [2, 3, 5, 1, 4, 6], [3, 5, 2, 1, 4, 6], [5, 2, 1, 3, 4, 6], [2, 1, 5, 3, 4, 6], [1, 2, 5, 3, 4, 6], [2, 5, 1, 3, 4, 6], [5, 1, 2, 3, 4, 6], [1, 5, 2, 3, 4, 6], [5, 2, 3, 1, 4, 6], [2, 5, 3, 1, 4, 6], [5, 3, 1, 2, 4, 6], [3, 1, 5, 2, 4, 6],

True

In [124]:
print(bell_ringer_algoritmo(3))

[[3, 2, 1], [2, 1, 3], [1, 3, 2], [3, 1, 2], [1, 2, 3], [2, 3, 1]]


In [137]:
window = 4
perm = window + 1

comprobar_u_ciclo(generador_u_ciclo(5, bell_ringer_algoritmo), 3, 2, relacion_equivalencia_shorthand)

[[2, 1], [1, 2]]
[3, 2, 1, 3, 1, 2]


True